# lstm 生成模型  anna

可以参照官方文档  这个也是生成模型   
https://tensorflow.google.cn/tutorials/text/text_generation

## 1数据处理

In [1]:
import time
from collections import namedtuple

import numpy as np

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

In [17]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
print(gpus, cpus)

[] [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [18]:
with open('../data/anna.txt', 'r') as f:
    text=f.read()
print(len(text))
text=text[:int(len(text)/10)]
print(len(text))

1985223
198522


In [19]:
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [20]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [21]:
encoded[:100]

array([17, 26, 20, 46, 32, 25, 62,  8,  3, 58, 58, 58, 16, 20, 46, 46, 15,
        8, 39, 20, 24, 40,  0, 40, 25, 29,  8, 20, 62, 25,  8, 20,  0,  0,
        8, 20,  0, 40, 55, 25,  4,  8, 25, 41, 25, 62, 15,  8, 23, 57, 26,
       20, 46, 46, 15,  8, 39, 20, 24, 40,  0, 15,  8, 40, 29,  8, 23, 57,
       26, 20, 46, 46, 15,  8, 40, 57,  8, 40, 32, 29,  8,  7, 63, 57, 58,
       63, 20, 15, 68, 58, 58, 28, 41, 25, 62, 15, 32, 26, 40, 57])

In [22]:
len(vocab)

75

In [23]:
print(len(text))
print(len(encoded))

encoded_train=encoded[1:]
print(len(encoded_train))
encoded_train=np.append(encoded_train,0)
print(len(encoded_train))
encoded_train[:100]

198522
198522
198521
198522


array([26, 20, 46, 32, 25, 62,  8,  3, 58, 58, 58, 16, 20, 46, 46, 15,  8,
       39, 20, 24, 40,  0, 40, 25, 29,  8, 20, 62, 25,  8, 20,  0,  0,  8,
       20,  0, 40, 55, 25,  4,  8, 25, 41, 25, 62, 15,  8, 23, 57, 26, 20,
       46, 46, 15,  8, 39, 20, 24, 40,  0, 15,  8, 40, 29,  8, 23, 57, 26,
       20, 46, 46, 15,  8, 40, 57,  8, 40, 32, 29,  8,  7, 63, 57, 58, 63,
       20, 15, 68, 58, 58, 28, 41, 25, 62, 15, 32, 26, 40, 57, 56])

## 2模型构建

In [24]:
batch_size = 100         # Sequences per batch
num_steps = 100
lstm_size = 800         # Size of hidden layers in LSTMs
learning_rate = 0.002    # Learning rate
keep_prob = 0.5         # Dropout keep probability
vocab_size=len(vocab)

In [25]:
int(len(encoded)/(batch_size*num_steps))*(batch_size*num_steps)

190000

In [26]:
def reshap_onehot_arr(arr,time_steps,cls_num):
    arr=arr[:int(len(encoded)/(batch_size*num_steps))*(batch_size*num_steps)]
    arr=np.reshape(arr,(-1,time_steps))
    arr=tf.keras.utils.to_categorical(arr,cls_num,dtype='int')
    print(arr.shape)
    return arr

train_x=reshap_onehot_arr(encoded,num_steps,vocab_size)
train_y=reshap_onehot_arr(encoded_train,num_steps,vocab_size)

(1900, 100, 75)
(1900, 100, 75)


 https://huhuhang.com/post/machine-learning/lstm-return-sequences-state  解释两个重要参数

In [3]:
def build_model(batch_size,stateful_flag,return_sequences_flag):
    model = keras.Sequential([
        layers.LSTM(lstm_size,
#                     dropout=keep_prob,
#                     recurrent_dropout=keep_prob,
                    return_sequences=return_sequences_flag,
                    stateful=stateful_flag,
                   batch_input_shape=[batch_size,None,vocab_size]),
        layers.Dense(vocab_size,activation="softmax")

    
    ])
    return model


model=build_model(batch_size=batch_size,stateful_flag=True,return_sequences_flag=True)

NameError: name 'batch_size' is not defined

In [28]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate),
                 loss=keras.losses.CategoricalCrossentropy(),
                 metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (100, None, 800)          2803200   
_________________________________________________________________
dense (Dense)                (100, None, 75)           60075     
Total params: 2,863,275
Trainable params: 2,863,275
Non-trainable params: 0
_________________________________________________________________


In [30]:
import os
# 检查点保存至的目录
checkpoint_dir = './training_checkpoints/lstm_anna/'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "lstm_anna_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [39]:
history = model.fit(train_x, train_y, batch_size=batch_size, epochs=10,callbacks=[checkpoint_callback])
# 将整个模型保存为HDF5文件
model.save('./model/anna_keras_lstm_epochs2.h5')

Train on 1900 samples
Epoch 1/10
1900/1900 [==============================] - 61s 32ms/sample - loss: 1.9599 - accuracy: 0.4338
Epoch 2/10
1900/1900 [==============================] - 61s 32ms/sample - loss: 1.9036 - accuracy: 0.4493
Epoch 3/10
1900/1900 [==============================] - 62s 33ms/sample - loss: 1.8602 - accuracy: 0.4587
Epoch 4/10
1900/1900 [==============================] - 62s 33ms/sample - loss: 1.8202 - accuracy: 0.4704
Epoch 5/10
1900/1900 [==============================] - 62s 33ms/sample - loss: 1.7811 - accuracy: 0.4809
Epoch 6/10
1900/1900 [==============================] - 62s 33ms/sample - loss: 1.7450 - accuracy: 0.4904
Epoch 7/10
1900/1900 [==============================] - 62s 33ms/sample - loss: 1.7079 - accuracy: 0.5004
Epoch 8/10
1900/1900 [==============================] - 62s 32ms/sample - loss: 1.6671 - accuracy: 0.5117
Epoch 9/10
1900/1900 [==============================] - 62s 32ms/sample - loss: 1.6265 - accuracy: 0.5237
Epoch 10/10
1900/1900 [=

In [40]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/lstm_anna/lstm_anna_10'

In [41]:
model_pre =build_model(batch_size=1,stateful_flag=True,return_sequences_flag=False)
model_pre.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) #只加载最后一次的weight
model_pre.build(tf.TensorShape([1,None,vocab_size]))
model_pre.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (1, 800)                  2803200   
_________________________________________________________________
dense_2 (Dense)              (1, 75)                   60075     
Total params: 2,863,275
Trainable params: 2,863,275
Non-trainable params: 0
_________________________________________________________________


In [42]:
a=np.array([1,2,3])
a=a.reshape([1,-1,1])
print(a)
a=tf.keras.utils.to_categorical(a,vocab_size,dtype='float32')
a

[[[1]
  [2]
  [3]]]


array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.,

In [45]:
test_predict_res = model_pre(a)
print(test_predict_res)
sum(test_predict_res.numpy().flatten())

tf.Tensor(
[[0.00387782 0.00355583 0.01432575 0.03160315 0.01879684 0.02237747
  0.00575899 0.0288036  0.00415679 0.02998908 0.01219524 0.00915078
  0.01883724 0.00723327 0.02751429 0.0326223  0.00659324 0.0050552
  0.01413766 0.00802052 0.009037   0.00329559 0.0085423  0.00902359
  0.00354656 0.00900195 0.00137365 0.00551215 0.01088765 0.0070126
  0.00450935 0.02868954 0.00686476 0.00442277 0.00793611 0.02432618
  0.01993652 0.0193146  0.01064909 0.00216354 0.02402181 0.00458153
  0.00907346 0.02575792 0.01002721 0.01868439 0.00286023 0.00877257
  0.00915062 0.01241996 0.00579801 0.014286   0.01473222 0.00993283
  0.01437463 0.05806882 0.00682235 0.00163117 0.02170659 0.01180592
  0.02033466 0.02918465 0.00502624 0.00554483 0.00932612 0.00893214
  0.00709701 0.00959793 0.00777258 0.00645924 0.00599416 0.00825742
  0.02460239 0.03221976 0.03449202]], shape=(1, 75), dtype=float32)


0.9999999202555045

In [87]:
pre_distrubtion = test_predict_res.numpy().flatten()
print(max(pre_distrubtion))
print(tf.argmax(pre_distrubtion))
np.random.choice(np.array(range(pre_distrubtion.shape[0])), 20, replace=True, p=pre_distrubtion)


0.058068816
tf.Tensor(55, shape=(), dtype=int64)


array([55, 58, 35, 35, 55, 73,  5, 73, 14, 28, 55, 37, 61, 17, 55, 61,  9,
       35, 65, 72])

In [111]:
def generate_text(model, start_string):
    # 评估步骤（用学习过的模型生成文本）

    # 要生成的字符个数
    num_generate = 100

    # 将起始字符串转换为数字（向量化）
    input_eval = np.array([vocab_to_int[s] for s in start_string])
    print(input_eval)
    input_eval = input_eval.reshape([1,-1,1])
    input_eval=tf.keras.utils.to_categorical(input_eval,vocab_size,dtype='float32')
    print("input_eval",input_eval.shape)

    # 空字符串用于存储结果
    text_generated = []

    # 低温度会生成更可预测的文本
    # 较高温度会生成更令人惊讶的文本
    # 可以通过试验以找到最好的设定
    temperature = 1.0

    # 这里批大小为 1
    model.reset_states()
    for i in range(num_generate):
        
        predictions = model(input_eval)
        print("predictions",predictions.shape)        
#         predicted_id = tf.argmax(predictions.numpy().flatten()).numpy()
#         print("predicted_id",predicted_id)
        predictions_distr = predictions.numpy().flatten()
        index_list = np.array(range(predictions_distr.shape[0]))
        predicted_id = np.random.choice(index_list, 1, replace=True, p=predictions_distr)
        pre_char= int_to_vocab[list(predicted_id)[0]]
        print("predicted_char",pre_char)
        text_generated.append(pre_char)
        
        input_eval = np.array([vocab_to_int[s] for s in pre_char])
        input_eval = input_eval.reshape([1,-1,1])
        input_eval=tf.keras.utils.to_categorical(input_eval,vocab_size,dtype='float32')

    return (start_string + ''.join(text_generated))

generate_text(model_pre,"am")

[20 24]
input_eval (1, 2, 75)
predictions (1, 75)
predicted_char p
predictions (1, 75)
predicted_char o
predictions (1, 75)
predicted_char i
predictions (1, 75)
predicted_char n
predictions (1, 75)
predicted_char g
predictions (1, 75)
predicted_char 

predictions (1, 75)
predicted_char a
predictions (1, 75)
predicted_char t
predictions (1, 75)
predicted_char  
predictions (1, 75)
predicted_char o
predictions (1, 75)
predicted_char w
predictions (1, 75)
predicted_char e
predictions (1, 75)
predicted_char ,
predictions (1, 75)
predicted_char  
predictions (1, 75)
predicted_char f
predictions (1, 75)
predicted_char o
predictions (1, 75)
predicted_char r
predictions (1, 75)
predicted_char  
predictions (1, 75)
predicted_char s
predictions (1, 75)
predicted_char i
predictions (1, 75)
predicted_char e
predictions (1, 75)
predicted_char q
predictions (1, 75)
predicted_char u
predictions (1, 75)
predicted_char e
predictions (1, 75)
predicted_char s
predictions (1, 75)
predicted_char  
predicti

'ampoing\nat owe, for sieques expriteron pratectly to nature, anat her idnep, and thouth down on enre. S'